In [1]:
!wget https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

--2025-07-13 16:41:20--  https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132013 (129K) [text/html]
Saving to: ‘individual+household+electric+power+consumption’

individual+househol 100%[===================>] 128.92K  --.-KB/s    in 0.08s   

2025-07-13 16:41:21 (1.56 MB/s) - ‘individual+household+electric+power+consumption’ saved [132013/132013]



In [4]:
!unzip /content/individual+household+electric+power+consumption.zip

Archive:  /content/individual+household+electric+power+consumption.zip
  inflating: household_power_consumption.txt  


In [5]:
!head /content/household_power_consumption.txt

Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3
16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000
16/12/2006;17:25:00;5.360;0.436;233.630;23.000;0.000;1.000;16.000
16/12/2006;17:26:00;5.374;0.498;233.290;23.000;0.000;2.000;17.000
16/12/2006;17:27:00;5.388;0.502;233.740;23.000;0.000;1.000;17.000
16/12/2006;17:28:00;3.666;0.528;235.680;15.800;0.000;1.000;17.000
16/12/2006;17:29:00;3.520;0.522;235.020;15.000;0.000;2.000;17.000
16/12/2006;17:30:00;3.702;0.520;235.090;15.800;0.000;1.000;17.000
16/12/2006;17:31:00;3.700;0.520;235.220;15.800;0.000;1.000;17.000
16/12/2006;17:32:00;3.668;0.510;233.990;15.800;0.000;1.000;17.000


In [6]:
import sqlite3
import pandas as pd
import csv

In [11]:
data=pd.read_csv("/content/household_power_consumption.txt")

In [14]:
df=pd.read_csv("/content/household_power_consumption.txt", sep=';', low_memory=False)

In [15]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [16]:
# connect to sql and create a database in sql
cnt=sqlite3.connect("household_power_consumption.db")

In [33]:
# create a table with the data for the database
df.to_sql("household_power_consn1umption", cnt, index=False, dtype={
    'Date': 'TEXT',
    'Time': 'TEXT',
    'Global_active_power': 'REAL',
    'Global_reactive_power': 'REAL',
    'Voltage': 'REAL',
    'Global_intensity': 'REAL',
    'Sub_metering_1': 'REAL',
    'Sub_metering_2': 'REAL',
    'Sub_metering_3': 'REAL'
})

2075259

In [34]:
#create querries using read_sql
query = "PRAGMA table_info(household_power_consn1umption)"
result = pd.read_sql(query,cnt)
print(result)

   cid                   name  type  notnull dflt_value  pk
0    0                   Date  TEXT        0       None   0
1    1                   Time  TEXT        0       None   0
2    2    Global_active_power  REAL        0       None   0
3    3  Global_reactive_power  REAL        0       None   0
4    4                Voltage  REAL        0       None   0
5    5       Global_intensity  REAL        0       None   0
6    6         Sub_metering_1  REAL        0       None   0
7    7         Sub_metering_2  REAL        0       None   0
8    8         Sub_metering_3  REAL        0       None   0


In [37]:
# data dimensions
query = """
SELECT 'rows' AS dimension, COUNT(*) AS count FROM household_power_consn1umption
UNION ALL
SELECT 'columns' AS dimension, COUNT(*) AS count FROM pragma_table_info('household_power_consn1umption');
"""
result = pd.read_sql(query,cnt)
print(result)

  dimension    count
0      rows  2075259
1   columns        9


In [39]:
# average consumption

query = "SELECT AVG(global_active_power) AS average_power FROM household_power_consn1umption;"
result = pd.read_sql(query,cnt)
print(result)

   average_power
0        1.07795


In [50]:
#check for null values across all the columns
query = """
SELECT
    SUM(CASE WHEN Date IS NULL THEN 1 ELSE 0 END) AS null_date,
    SUM(CASE WHEN Time IS NULL THEN 1 ELSE 0 END) AS null_time,
    SUM(CASE WHEN global_active_power IS NULL THEN 1 ELSE 0 END) AS null_global_active_power,
    SUM(CASE WHEN global_reactive_power IS NULL THEN 1 ELSE 0 END) AS null_global_reactive_power,
    SUM(CASE WHEN Voltage IS NULL THEN 1 ELSE 0 END) AS null_voltage,
    SUM(CASE WHEN global_intensity IS NULL THEN 1 ELSE 0 END) AS null_global_intensity,
    SUM(CASE WHEN sub_metering_1 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_1,
    SUM(CASE WHEN sub_metering_2 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_2,
    SUM(CASE WHEN sub_metering_3 IS NULL THEN 1 ELSE 0 END) AS null_sub_metering_3
FROM household_power_consumption;
"""
result = pd.read_sql(query,cnt)
print(result)

   null_date  null_time  null_global_active_power  null_global_reactive_power  \
0          0          0                         0                           0   

   null_voltage  null_global_intensity  null_sub_metering_1  \
0             0                      0                    0   

   null_sub_metering_2  null_sub_metering_3  
0                    0                25979  


In [45]:
#running querries using cursor object

In [51]:
#open text file and read all rows
with open('/content/household_power_consumption.txt','r') as file:
  reader = csv.reader(file,delimiter=';')

  #skip header row
  header = next(reader)

  #replace missing values with "None" and prepare the data
  data = [
      [None if value == '?' else value for value in row]
      for row in reader
  ]

#use "executemany" to insert all rows at once
insert_query = """
INSERT INTO household_power_consumption (
    Date, Time, global_active_power, global_reactive_power,
    Voltage, global_intensity, sub_metering_1, sub_metering_2, sub_metering_3
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);
"""
cursor = cnt.cursor()
cursor.executemany(insert_query,data)

#commit changes
cnt.commit()

In [49]:
# running Queries on the SQLite Database with the cursor Object

In [52]:
#query database to retrieve column names
query = "PRAGMA table_info(household_power_consumption);"
cursor.execute(query)
result = cursor.fetchall()

for column in result:
  print(column)

(0, 'Date', 'TEXT', 0, None, 0)
(1, 'Time', 'TEXT', 0, None, 0)
(2, 'Global_active_power', 'REAL', 0, None, 0)
(3, 'Global_reactive_power', 'REAL', 0, None, 0)
(4, 'Voltage', 'REAL', 0, None, 0)
(5, 'Global_intensity', 'REAL', 0, None, 0)
(6, 'Sub_metering_1', 'REAL', 0, None, 0)
(7, 'Sub_metering_2', 'REAL', 0, None, 0)
(8, 'Sub_metering_3', 'REAL', 0, None, 0)


In [54]:
#finds the 5 times of day with the highest total power usage across all days in the dataset
query = """
SELECT Time, SUM(global_active_power) AS total_power
FROM household_power_consumption
GROUP BY Time
ORDER BY total_power DESC
LIMIT 5;
"""

cursor.execute(query)
results = cursor.fetchall()

for row in results:
  print(row)

('20:50:00', 5714.364000000006)
('20:49:00', 5713.552000000005)
('20:51:00', 5710.604000000004)
('20:52:00', 5688.560000000005)
('20:53:00', 5676.796000000004)


In [55]:

#close connection after finishing queries
cnt.close()

